### Определение расстояния до впереди идущего автомобиля на основе изображений 🚗

<h1><center>Задача</center></h1>
Разработать алгоритм, позволяющий определить дистанцию до впереди идущего автомобиля,используя для этого датасет фотографий автомобилей с разного расстояния.Впоследствии этот алгоритм может быть использован в системах навигации для предупреждения об опасном сближении и для контроля за соблюдением дистанции.


**Краткое описание решения:**

    ● Изображения преобразованы в черно-белое;
    ● Использовалась предобученная модель YOLOv5l6 для детекции автомобилей;
    ● Полученные координаты были признаками для регрессора;
    ● Из полученных координат созданы доп признаки;
    ● Поиск оптимального регрессора.

In [ ]:
import pandas as pd
import math
import os
import cv2
import time
from math import sqrt
from sklearn.metrics import mean_squared_error,r2_score
from PIL import Image
import numpy as np
import pillow_heif
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
from sklearn.linear_model import Ridge, Lasso,ElasticNet,LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import (BaggingRegressor,ExtraTreesRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor,RandomForestRegressor,StackingRegressor)
import pathlib
from PIL import Image, ImageFilter  
import warnings
%matplotlib

warnings.filterwarnings("ignore")
import torch


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)
model.cuda()# подключил графический процессор
model.classes = [2]# выбрал класс авто

In [ ]:
sub = pd.read_csv('sample_solution.csv', sep=';', index_col=None)

In [ ]:
train_labels_df = pd.read_csv(r'C:\work\Чемпионат_ульяновск\Ульяновск\train_dataset_train/train.csv', sep=';', index_col=None)

In [ ]:
path_train = 'participants/train/'
path_test = 'participants/test/'

In [ ]:
test_img_names = set(os.listdir('participants/test'))
train_img_names = set(os.listdir('participants/train'))

In [ ]:
def increase_brightness(img, value):# фукнция яркости изображения
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
train_data = [] 
for img_name in tqdm(train_img_names):
    expansion=(pathlib.Path(img_name).suffix)# вывел расширение изображения
    if 'heic' in img_name:
        heif_file = pillow_heif.read_heif(path_train+ img_name)
        img= Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
        img = img.filter(ImageFilter.SMOOTH)
        img = img.filter(ImageFilter.EDGE_ENHANCE)
        img = img.convert('L')
        shape_img=(3024,4032)
        img=np.asarray(img)
        results = model(img,augment=True)
        results = results.pandas().xyxy[0].to_dict(orient="records")
        for result in results:
               
            con = result['confidence']
            cs = result['class']
            x1 = int(result['xmin'])
            y1 = int(result['ymin'])
            x2 = int(result['xmax'])
            y2 = int(result['ymax'])
            name = result['name']
            image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
            train_data.append(image)
           
    else:
        img = cv2.imread(path_train+img_name)
     
        sharped_img = cv2.medianBlur (img, 7)
        frame = increase_brightness(sharped_img, value=50)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        results = model(img,augment=True)
        results = results.pandas().xyxy[0].to_dict(orient="records")
        shape_img=str(img.shape)
        if shape_img=='(3024, 4032)': 
            for result in results:
               
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])
                shape_img=str(img.shape)
                name = result['name']
                image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
                train_data.append(image)
        else:
            imag = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            results = model(imag,augment=True)
            results = results.pandas().xyxy[0].to_dict(orient="records")
            for result in results:
               
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])
                shape_img=str(imag.shape)
                name = result['name']       
                image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
                train_data.append(image)

In [ ]:
# Создал датафрейм с признаками тренировочных изображений
train_data_df = pd.DataFrame(train_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class','name','shape_img','expansion'])

In [ ]:
test_data = [] 
for img_name in tqdm(test_img_names):
    expansion=(pathlib.Path(img_name).suffix)
    if 'heic' in img_name:
        heif_file = pillow_heif.read_heif(path_test+ img_name)
        img= Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
        
        img = img.convert('L')
        shape_img=(3024,4032)
        img=np.asarray(img)
        results = model(img,augment=True)
        results = results.pandas().xyxy[0].to_dict(orient="records")
        for result in results:
               
            con = result['confidence']
            cs = result['class']
            x1 = int(result['xmin'])
            y1 = int(result['ymin'])
            x2 = int(result['xmax'])
            y2 = int(result['ymax'])
            name = result['name']
            image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
            test_data.append(image)  
    else:
        img = cv2.imread(path_test+img_name)
     
        sharpen_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

       
        sharped_img = cv2.filter2D(img, -1, sharpen_filter)
        frame = increase_brightness(sharped_img, value=30)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        results = model(img,augment=True)
        results = results.pandas().xyxy[0].to_dict(orient="records")
        shape_img=str(img.shape)
        if shape_img=='(3024, 4032)':   
            for result in results:
               
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])
                shape_img=str(img.shape)
                name = result['name']
           
                image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
        
                test_data.append(image)
        else:
            imag = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            results = model(imag,augment=True)
            results = results.pandas().xyxy[0].to_dict(orient="records")
            for result in results:
               
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])
                shape_img=str(imag.shape)
                name = result['name']
           
                image=[img_name,x1,y1,x2,y2,con,cs,name,shape_img,expansion]
        
                test_data.append(image)

In [ ]:
# Создал датафрейм с признаками тестовочных изображений
test_data_df = pd.DataFrame(test_data, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'class','name','shape_img','expansion'])

In [ ]:
# Создана функция с помощью которой остались автомобили которые по центру изображения, а так же доп признаки

In [ ]:
def transform_feature(df,img_h, img_w):
    df['x_center'] = (df['x_min'] + df['x_max'])/2
    df['x_center_total']=abs((img_w/2) -df['x_center'])
    feature_label_=df.image_name.unique().tolist()
    data=pd.DataFrame()
    for i in feature_label_:# т.к. yolov5l6 нашла все авто на изображениях, я отфильтровал по центру
        df_img= df.loc[df['image_name']==i]
        df_x_central=df_img[(df_img['x_center_total'] == df_img['x_center_total'].min())] 
        data=data.append(df_x_central)
   
    data['pixel_distance'] = ((data['x_max'] - data['x_min']) ** 2 + (data['y_max'] - data['y_min']) ** 2)
    data['pixel_distance_sqrt']=data['pixel_distance'].apply(np.sqrt)
    
    data['y_center'] = (data['y_min']+ data['y_max'])/2
  
    data['y_center_total']=abs((img_h/2)- data['y_center'])
    data['w'] = data['x_max'] - data['x_min']
    data['h'] = data['y_max'] - data['y_min']
    data['x_center_w'] = data['x_center']/img_w
    data['w_'] = data['w']/img_w
    data['y_center_h'] = data['y_center']/img_h
    data['h_'] = data['h']/img_h
    data['s']=((data['y_max']-data['y_min'])*(data['x_max']-data['x_min']))# площадь рамки
    data['p']=(((data['y_max']-data['y_min'])+(data['x_max']-data['x_min'])))*2  # периметр рамки
    data['corner']=np.rad2deg(np.arctan((data['y_max']-data['y_min'])/(data['x_max']-data['x_min'])))
    data['corner_img']=np.rad2deg(np.arctan((img_h-data['y_center'])/(img_w-data['x_center'])))
    return data

In [ ]:
data_train=transform_feature(train_data_df,3024, 4032)
data_test=transform_feature(test_data_df,3024,4032)

In [ ]:
distance=dict(zip(train_labels_df.image_name,train_labels_df.distance))
data_train['distance'] = data_train['image_name'].apply(lambda x: distance.get(x, 0))

In [ ]:
data_train.expansion.value_counts()

In [ ]:
data_test.expansion.value_counts()

In [ ]:
corr = data_train[['x_min', 'y_min', 'x_max','y_max', 'conf','pixel_distance_sqrt','s','p','y_center','distance']].corr()
corr.style.background_gradient(cmap='coolwarm',axis=0)

In [ ]:
data_train=data_train.drop_duplicates(subset ='distance')

In [ ]:
X_train_data=data_train.drop(['class','distance','shape_img','image_name','name','pixel_distance_sqrt','expansion'],axis=1)
X_valid=data_test.drop(['class','shape_img','image_name','name','pixel_distance_sqrt','expansion'],axis=1)
targets=data_train['distance'].values


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_data, targets, test_size=0.05, random_state=42)

In [ ]:
models = [ ['Lasso: ', Lasso()],
           ['Ridge: ', Ridge()],
           ['KNeighborsRegressor: ',  KNeighborsRegressor()],
           ['BaggingRegressor: ',  BaggingRegressor(n_jobs=-1)],
           ['RandomForest ', RandomForestRegressor()],
           ['ExtraTreeRegressor :',ExtraTreesRegressor(n_jobs=-1)],
           ['GradientBoostingRegressor: ', GradientBoostingRegressor()] ,
           ['XGBRegressor: ', XGBRegressor(n_jobs=-1)] ,
           ['DecisionTreeRegressor: ', DecisionTreeRegressor()] ,
           ['LGBM: ', LGBMRegressor()] ,
           ['CatBoostRegressor: ', CatBoostRegressor()] ,
           ['HistGradientBoostingRegressor: ', HistGradientBoostingRegressor()] ,
           ['ElasticNet: ', ElasticNet()] ,
           ['StackingRegressor: ', StackingRegressor([('Extra',ExtraTreesRegressor() ),('GradientBoostingRegressor',GradientBoostingRegressor() ), ('Random',RandomForestRegressor())], n_jobs=-1)] ,
           
         ]

In [ ]:
model_train = []
for name,train_model in models :
    train_model_data = {}
    train_model.random_state = 42
    train_model_data["Регрессор"] = name
    start = time.time()
    train_model.fit(X_train,y_train)
    end = time.time()
    train_model_data["Время обучения"] = end - start
    train_model_data["Train_R2_Score"] = r2_score(y_train,train_model.predict(X_train))
    train_model_data["Test_R2_Score"] = r2_score(y_val,train_model.predict(X_val))
    train_model_data["Test_RMSE_Score"] = sqrt(mean_squared_error(y_val,train_model.predict(X_val)))
    model_train.append(train_model_data)

In [ ]:
df = pd.DataFrame(model_train)
df=df.sort_values(by=['Test_R2_Score'],ascending=False)
df

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{
              'max_depth': [10,80, 150, 200,250],
              'n_estimators' : [100,150,200,250,300],
              'max_features': ["auto", "sqrt", "log2"],
              'ccp_alpha': [0.001,0.01,0.1],
         
            }]
reg = ExtraTreesRegressor()

grid_search = GridSearchCV(estimator = reg, param_grid = param_grid, cv = 7, n_jobs=-1, scoring='r2' , verbose=2)
grid_search.fit(X_train_data, targets)

In [ ]:
grid_search.best_params_

In [ ]:
params = [{'n_estimators': [10,30,40, 100,200,300, 400],
           'learning_rate': [0.05, 0.1, 0.5],
           'min_samples_split': [8, 12],
           'min_samples_leaf': [2, 3, 4],
           'max_depth': [2, 3, 4]}]

gboost_reg = GradientBoostingRegressor()

cv_scheme = KFold(n_splits=7, shuffle=True, random_state=1)
cv = GridSearchCV(estimator=gboost_reg, param_grid=params, scoring='r2',
                  cv=cv_scheme, return_train_score=True, n_jobs=-1,verbose=2)

cv.fit(X_train_data, targets)

In [ ]:
cv.best_params_

In [ ]:
feature_cols=X_train_data.columns

In [ ]:
cv = KFold(n_splits=15, random_state=7777, shuffle=True)
val_preds  = np.zeros(len(data_train))
train_preds = np.zeros(len(data_train))
val_preds_  = np.zeros(len(data_train))
train_preds_ = np.zeros(len(data_train))
models = []
tree_params = {      'ccp_alpha': 0.001,
                     'max_depth': 200,
                     'max_features': 'sqrt',
                     'n_estimators': 250}

gradient_params = {  'learning_rate': 0.05,
                     'max_depth': 4,
                     'min_samples_leaf': 2,
                     'min_samples_split': 8,
                     'n_estimators': 300}

for fold_, (train_idx, val_idx) in enumerate(cv.split(data_train, targets), 1):
    print(f"Training with fold {fold_} started")
    Extra_model = ExtraTreesRegressor(**tree_params)
    Gradient_model = GradientBoostingRegressor(**gradient_params)
    train, val = data_train.iloc[train_idx], data_train.iloc[val_idx]
    
    Extra_model.fit(train[feature_cols], train.distance.values)
    Gradient_model.fit(train[feature_cols], train.distance.values)
    
    val_preds[val_idx] = Extra_model.predict(val[feature_cols])
    val_preds_[val_idx] = Gradient_model.predict(val[feature_cols])
    train_preds[train_idx] += Extra_model.predict(train[feature_cols]) / (cv.n_splits-1)
    train_preds_[train_idx] += Gradient_model.predict(train[feature_cols]) / (cv.n_splits-1)
    models.append(Extra_model)
    models.append(Gradient_model)

    print('ExtraTrees', r2_score(val_preds[val_idx],val.distance.values))
    print('GradientBoosting', r2_score(val_preds_[val_idx],val.distance.values))
    print(f"Training with fold {fold_} completed")

In [ ]:
print("Train Extra : ", r2_score(targets, train_preds))
print("Train GradientBoosting : ", r2_score(targets, train_preds_))

In [ ]:
print("Test Extra: ", r2_score(targets, val_preds))
print("Test GradientBoosting: ", r2_score(targets, val_preds_))

In [ ]:
import tqdm
score = np.zeros(len(data_test))

for model in tqdm.tqdm_notebook(models):
    score += model.predict(data_test[feature_cols]) / len(models)
    
submission = pd.DataFrame({
    "image_name" : data_test["image_name"].values,
    "distance" : score
}) 

submission.to_csv("submission.csv", sep=';', index=False)

In [ ]:
submission